# OHCL - Analytics

### Import Library

In [118]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [119]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [120]:
data = dataset.copy()

In [121]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2024-09-18 14:15:00,1310.7,1313.5,1308.7,1312.8,8783
2024-09-18 14:20:00,1312.8,1313.2,1310.8,1311.3,6209
2024-09-18 14:25:00,1311.1,1313.3,1311.1,1311.2,5452


In [122]:
def prepare_data(htd):
    #
    htd = htd[(htd.Volume >= 300) & (htd.High != htd.Low)]
    #
    htd['ibs'] = htd.apply(
        lambda x: (x["Close"] - x["Low"]) / (x["High"] - x["Low"]), axis=1)
    htd['body_rate'] = htd.apply(
        lambda x: (x["Close"] - x["Open"]) / (x["High"] - x["Low"]), axis=1)
    htd["ma_20_vol"] = htd["Volume"].rolling(20).mean()
    htd['vol_rate'] = htd.apply(
        lambda x: round(x["Volume"] / x["ma_20_vol"], 2), axis=1)
    #
    htd['min_OC'] = htd.apply(
        lambda r: min(r['Open'], r['Close']), axis=1)
    htd['max_OC'] = htd.apply(
        lambda r: max(r['Open'], r['Close']), axis=1)
    htd["max_3"] = htd["High"].rolling(3).max()
    htd["min_3"] = htd["Low"].rolling(3).min()
    htd["max_10"] = htd["High"].rolling(10).max()
    htd["min_10"] = htd["Low"].rolling(10).min()
    htd['in_position'] = htd.apply(
        lambda x: check_position(x), axis=1)
    #
    htd['high_rate'] = 100 * (htd.max_10 - htd.Close) / htd.max_10
    htd['low_rate'] = 100 * (htd.Close - htd.min_10) / htd.min_10
    return htd


def check_position(row):
    if row["min_OC"] - row["min_3"] <= 3 or row['max_3'] - row['max_OC'] <= 3:
        return 1
    elif 100 * row.name.hour + row.name.minute == 900:
        return 1
    else:
        return 0

In [123]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

,Open,High,Low,Close,Volume,ibs,body_rate,ma_20_vol,vol_rate,min_OC,max_OC,max_3,min_3,max_10,min_10,in_position,high_rate,low_rate
Date,,,,,,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,0.285714,-0.571429,NaN,NaN,943.1,943.5,NaN,NaN,NaN,NaN,1,NaN,NaN
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,0.666667,0.333333,NaN,NaN,943.1,943.3,NaN,NaN,NaN,NaN,0,NaN,NaN
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,0.714286,-0.142857,NaN,NaN,943.1,943.2,943.6,942.6,NaN,NaN,1,NaN,NaN
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,0.375000,-0.625000,NaN,NaN,942.6,943.1,943.5,942.3,NaN,NaN,1,NaN,NaN
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,1.000000,0.846154,NaN,NaN,942.6,943.7,943.7,942.3,NaN,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-18 14:05:00,1311.1,1311.3,1308.9,1311.0,13369,0.875000,-0.041667,4974.95,2.69,1311.0,1311.1,1314.4,1308.9,1314.5,1308.9,1,0.266261,0.160440
2024-09-18 14:10:00,1310.9,1311.9,1310.4,1310.8,4545,0.266667,-0.066667,5007.25,0.91,1310.8,1310.9,1314.4,1308.9,1314.4,1308.9,1,0.273889,0.145160
2024-09-18 14:15:00,1310.7,1313.5,1308.7,1312.8,8783,0.854167,0.437500,5338.40,1.65,1310.7,1312.8,1313.5,1308.7,1314.4,1308.7,1,0.121729,0.313288


In [124]:
prepared_data[prepared_data.in_position == 1]

,Open,High,Low,Close,Volume,ibs,body_rate,ma_20_vol,vol_rate,min_OC,max_OC,max_3,min_3,max_10,min_10,in_position,high_rate,low_rate
Date,,,,,,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,0.285714,-0.571429,NaN,NaN,943.1,943.5,NaN,NaN,NaN,NaN,1,NaN,NaN
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,0.714286,-0.142857,NaN,NaN,943.1,943.2,943.6,942.6,NaN,NaN,1,NaN,NaN
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,0.375000,-0.625000,NaN,NaN,942.6,943.1,943.5,942.3,NaN,NaN,1,NaN,NaN
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,1.000000,0.846154,NaN,NaN,942.6,943.7,943.7,942.3,NaN,NaN,1,NaN,NaN
2018-08-13 09:25:00,943.8,945.9,943.7,945.3,2469,0.727273,0.681818,NaN,NaN,943.8,945.3,945.9,942.3,NaN,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-18 14:05:00,1311.1,1311.3,1308.9,1311.0,13369,0.875000,-0.041667,4974.95,2.69,1311.0,1311.1,1314.4,1308.9,1314.5,1308.9,1,0.266261,0.160440
2024-09-18 14:10:00,1310.9,1311.9,1310.4,1310.8,4545,0.266667,-0.066667,5007.25,0.91,1310.8,1310.9,1314.4,1308.9,1314.4,1308.9,1,0.273889,0.145160
2024-09-18 14:15:00,1310.7,1313.5,1308.7,1312.8,8783,0.854167,0.437500,5338.40,1.65,1310.7,1312.8,1313.5,1308.7,1314.4,1308.7,1,0.121729,0.313288


In [125]:
prepared_data[(prepared_data.index > '2024-09-18 08:05:00') & (prepared_data.in_position == 1)]

,Open,High,Low,Close,Volume,ibs,body_rate,ma_20_vol,vol_rate,min_OC,max_OC,max_3,min_3,max_10,min_10,in_position,high_rate,low_rate
Date,,,,,,,,,,,,,,,,,,
2024-09-18 09:00:00,1305.0,1305.7,1304.0,1304.0,4270,0.000000,-0.588235,5536.80,0.77,1304.0,1305.0,1305.7,1296.2,1305.7,1287.1,1,0.130198,1.313029
2024-09-18 09:05:00,1304.5,1305.4,1304.1,1304.8,1994,0.538462,0.230769,5539.35,0.36,1304.5,1304.8,1305.7,1298.1,1305.7,1287.1,1,0.068929,1.375185
2024-09-18 09:10:00,1305.0,1305.2,1304.6,1305.2,895,1.000000,0.333333,5472.45,0.16,1305.0,1305.2,1305.7,1304.0,1305.7,1288.0,1,0.038294,1.335404
2024-09-18 09:15:00,1305.2,1305.4,1304.2,1304.3,2945,0.083333,-0.750000,5513.55,0.53,1304.3,1305.2,1305.4,1304.1,1305.7,1289.7,1,0.107222,1.132046
2024-09-18 09:20:00,1304.2,1304.2,1303.2,1303.5,2921,0.300000,-0.700000,5362.60,0.54,1303.5,1304.2,1305.4,1303.2,1305.7,1290.3,1,0.168492,1.023018
2024-09-18 09:25:00,1303.5,1304.6,1303.5,1304.2,1447,0.636364,0.636364,5241.05,0.28,1303.5,1304.2,1305.4,1303.2,1305.7,1290.8,1,0.114881,1.038116
2024-09-18 09:30:00,1304.2,1307.5,1303.6,1307.1,4492,0.897436,0.743590,5147.50,0.87,1304.2,1307.1,1307.5,1303.2,1307.5,1293.2,1,0.030593,1.074853
2024-09-18 09:35:00,1307.4,1307.4,1306.1,1306.4,3595,0.230769,-0.769231,5186.85,0.69,1306.4,1307.4,1307.5,1303.5,1307.5,1296.2,1,0.084130,0.786916
2024-09-18 09:40:00,1306.4,1307.5,1306.3,1306.6,2583,0.250000,0.166667,5027.80,0.51,1306.4,1306.6,1307.5,1303.6,1307.5,1298.1,1,0.068834,0.654803
